<a href="https://colab.research.google.com/github/atjoelpark/ml-disparities-mit/blob/master/pull_preprocessing/LCP_Pull_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pull and Extraction of LCP PMIDs

This notebook provides documentation and code for pulling metadata for a list of PMID IDs.

Reference: https://lcp.mit.edu/publications

In [119]:
# Customization
# Please enter in the pathway in your Google Drive (after /content/drive/) that you would like your files to be saved into
# Users will only need to modify this code then run all the cells in order
google_drive_url = ""

# Libraries and Mounting Google Drive

In [120]:
# Importing libraries
import numpy as np 
import pandas as pd 
import re 
import requests
from bs4 import BeautifulSoup

In [3]:
# Mounting Google Drive if using Google Drive
from google.colab import drive
drive.mount(f'/content/drive/{google_drive_url}')

Mounted at /content/drive/


## Defining Functions

In [121]:
# Installing E-utilities Entrez Direct
def e_utilities_install():
  """
  Installs e_utilities
  Reference: https://www.ncbi.nlm.nih.gov/books/NBK179288/
  """
  !curl -L https://www.ncbi.nlm.nih.gov/books/NBK179288/bin/install-edirect.sh > install-edirect.sh
  !bash install-edirect.sh -y
  !echo 'export PATH=\$PATH:\$HOME/edirect' >> $HOME/.bash_profile
  !rm install-edirect.sh

In [122]:
# Setting development environment for Selenium
def setup_dev_environment():
  """
  Installs chromium, driver and selenium
  Sets options to be headless
  Opens a website and prepares Selenium for use
  Returns: webdriver
  """

  # install chromium, its driver, and selenium
  !apt update
  !apt install chromium-chromedriver
  !pip install selenium
  !pip install webdriver_manager
  # set options to be headless, ..
  from selenium import webdriver
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  # open it, go to a website, and get results
  print("Chromium, Driver and Selenium successfully started....")
  return webdriver.Chrome(options=options)

In [123]:
# Defining Functions
def pull_pmid(year: int, wb) -> list:
  """ 
  This takes in a year as an argument and reads the PMID IDs for each year within https://lcp.mit.edu/publications

  Dependencies: Selenium, chromium-chromedriver, webdriver_manager, re
  @param year: This int contains the year to scrape from in https://lcp.mit.edu/publications
  @param wb: Passes in the webdriver for Selenium
  @return: a list of PMIDs
  @raise TypeError: raises an exception
  """
  try:
    # Uses Selenium to search by CSS
    URL = f'https://lcp.mit.edu/publications#P_{year}'
    wd.get(URL)
    _links = wd.find_elements_by_css_selector('.bib2xhtml a+ a')

    # Initiating an empty PMID list and appends to list with all PMID IDs
    _pmid_list = []
    for i in _links:
      tmp_search = re.findall(r'\(PMID:.*\)', i.text)
      if tmp_search:
        _pmid_list.append(tmp_search)

    # Flattends the list
    _pmid_list = [item for sublist in _pmid_list for item in sublist]

    # Extracts out only integers and removes text and special characters. 
    # Returns the list
    _pmid_list = [int(re.findall(r'\d+', i)[0]) for i in _pmid_list]
    return _pmid_list

  except TypeError as e:
    print("Error raised while pulling PMID...")
    print(e)

In [170]:
# Defining Functions
def pull_pmid_metadata(pmid: list) -> pd.DataFrame:
  """
  This is dependent on E-utilities. The input parameter is a list of PMIDs.
  The output is a Pandas DataFrame with the following columns:

  1. PMID
  2. PubMed Article Title
  3. DateCompleted (Year_Month_Day)
  4. Journal Title
  5. Publication Date (Year_Month_Day)
  6. Abstract
  7. Author FirstName_LastName_Affiliation (Note that that three values are
  separated by "_". If an author has affilitations to multiple institions, the
  institutions are separated by the character "/".)

  @param pmid: Takes a list of PMIDs produced by function pull_pmid
  @return: Returns a Pandas DataFrame
  @raise keyError: raises an exception
  """
  df = pd.DataFrame(columns=["PMID", "PubMed_Article_Title",
                             "Date_Completed", "Date_Revised","Journal_Title",
                             "Publication_Date", "Abstract", "AuthorFirstName_AuthorLastName_Affiliation"])
  for id, i in enumerate(pmid):
    _temp = f'''$HOME/edirect/efetch -db pubmed -id {i} -format xml \
| $HOME/edirect/xtract -pattern PubmedArticle -tab "|" -def "NULL" -sep "," -element MedlineCitation/PMID ArticleTitle \
-block DateCompleted -tab "|" -def "NULL" -sep "_" -element Year,Month,Day \
-block Article -tab "|" -def "NULL" -sep "_" -element Journal/Title \
-subset PubDate -tab "|" -def "NULL" -sep "_" -element Year,Month,Day \
-section Abstract -tab "|" -def "NULL" -element AbstractText \
-block Author -tab "/" -def "NULL" -sep "_" -element ForeName,LastName,Affiliation'''

    _result = !{_temp}
    _temp = _result[0].split("|")

    print(_temp)
    for _count, _value in enumerate(_temp):
      #df.iloc[id,_count] = _value
      print(_count, _value)

  #return df

# Main

In [125]:
%%time
# Install E-utilities, when prompted "Would you like to do that automatically now?" Please select 'y'.
e_utilities_install()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   665  100   665    0     0   1722      0 --:--:-- --:--:-- --:--:--  1718

Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "source ~/.bash_profile" >> $HOME/.bashrc

or manually edit the PATH variable assignment in your .bash_profile file.

Would you like to do that automatically now? [y/N]
y
OK, done.
CPU times: user 118 ms, sys: 37.6 ms, total: 156 ms
Wall time: 18.1 s


In [126]:
%%time 
# Sets up Web Driver
wd = setup_dev_environment()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [111]:
%%time
pmid_list_2020 = pull_pmid(2020, wd)

CPU times: user 653 ms, sys: 21.1 ms, total: 674 ms
Wall time: 9.18 s


In [171]:
%%time 
pull_pmid_metadata(pmid_list_2020)

['32612144', 'Real-world characterization of blood glucose control and insulin use in the intensive care unit.', '2020_12_04', 'Scientific reports', '2020_07_01', 'The heterogeneity of critical illness complicates both clinical trial design and real-world management. This complexity has resulted in conflicting evidence and opinion regarding the optimal management in many intensive care scenarios. Understanding this heterogeneity is essential to tailoring management to individual patients. Hyperglycaemia is one such complication in the intensive care unit (ICU), accompanied by decades of conflicting evidence around management strategies. We hypothesized that analysis of highly-detailed electronic medical record (EMR) data would demonstrate that patients vary widely in their glycaemic response to critical illness and response to insulin therapy. Due to this variability, we believed that hyper- and hypoglycaemia would remain common in ICU care despite standardised approaches to management

In [112]:
# Ignore below
pmid_list_2020

[32612144,
 31948262,
 32126097,
 32240233,
 32248145,
 32577533,
 32449686,
 32577534,
 32432708]